In [3]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Input, concatenate, GlobalAveragePooling2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

Using TensorFlow backend.


In [4]:
df = pd.read_csv('train.csv')
y = df['label'].to_numpy()
X = df.drop('label', axis = 1)
X = X.to_numpy().reshape(df.shape[0],28,28,1)

df_test = pd.read_csv('test.csv')
X_test = df_test.drop('id', axis=1).to_numpy().reshape(df_test.shape[0],28,28,1)

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255.,
                                   rotation_range=10,
                                   width_shift_range=0.25,
                                   height_shift_range=0.25,
                                   shear_range=0.1,
                                   zoom_range=0.25,
                                   horizontal_flip=False)

valid_datagen = ImageDataGenerator(rescale=1./255.)

In [6]:
nets = 15
models = [0] * nets

for i in range(nets):
    models[i] = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(64, kernel_size=5, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Conv2D(128, kernel_size=3, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(128, kernel_size=3, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(128, kernel_size=5, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Conv2D(256, kernel_size=3, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
    models[i].compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

KeyboardInterrupt: 

In [ ]:
lrs = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)


history = [0] * nets

for j in range(nets):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.1)
    history[j] = models[j].fit_generator(train_datagen.flow(X_train, y_train, batch_size=64),
                              steps_per_epoch=X_train.shape[0] // 64,
                              epochs=45,
                              validation_data=valid_datagen.flow(X_val, y_val),
                              callbacks=[lrs],
                              verbose=1)
    
    print("CNN ", j + 1, 'Epochs=45 Train Accuracy=', max(history[j].history['acc']), 'Validation Accuracy=', max(history[j].history['val_acc']))

In [ ]:
results = np.zeros((X_test.shape[0],10)) 
for j in range(nets):
    results = results + model[j].predict(X_test * 1.0 / 255.0)
y_pred = np.argmax(results, axis=1)

In [ ]:
answer = pd.DataFrame(y_pred, columns=['label'])
answer.index.name = 'id'
answer.to_csv('submission.csv')